In [1]:
# === Cell 1: Imports and setup ===
import os, glob, json, numpy as np, pandas as pd
import SimpleITK as sitk
from pathlib import Path
from tqdm import tqdm

# Define the structures to evaluate
STRUCTURES = [
    "mandible",
    "skull",
    "head",
    "teeth_lower",
    "teeth_upper",
    "sinus_frontal",
    "sinus_maxillary"
]

# Base data path
BASE_PATH = r"Z:\FacialDeformation_MPhys\paired_data\nina_abby"

# Where to save results
OUTDIR = Path(BASE_PATH) / "evaluation_results"
OUTDIR.mkdir(exist_ok=True)


In [ ]:
# === Cell 2: Generate perturbed CTs for new runs ===
import random

def add_noise_and_smooth(img, sigma_noise=20, smoothing_iterations=1):
    arr = sitk.GetArrayFromImage(img).astype(np.float32)
    noise = np.random.normal(0, sigma_noise, arr.shape)
    noisy = arr + noise
    noisy_img = sitk.GetImageFromArray(noisy)
    noisy_img.CopyInformation(img)
    return sitk.CurvatureFlow(noisy_img, timeStep=0.125, numberOfIterations=smoothing_iterations)

# Example usage: create 3 synthetic variants of each CT
src_cases = [d for d in Path(BASE_PATH).glob("UID*") if d.is_dir()]
num_runs = 3

for run_idx in range(1, num_runs+1):
    run_dir = Path(BASE_PATH) / f"TS_run{run_idx}_input"
    run_dir.mkdir(exist_ok=True)
    for case in tqdm(src_cases, desc=f"Preparing run{run_idx}"):
        ct_path = case / "CT_WHOLE_CNS_cropped.nii"   # adjust if your CT filenames differ
        if not ct_path.exists(): continue
        img = sitk.ReadImage(str(ct_path))
        perturbed = add_noise_and_smooth(img, sigma_noise=10*run_idx, smoothing_iterations=run_idx)
        out_path = run_dir / f"{case.name}_ct.nii.gz"
        sitk.WriteImage(perturbed, str(out_path))


Preparing run1:   0%|          | 0/9 [00:00<?, ?it/s]

In [ ]:
from totalsegmentator.python_api import totalsegmentator

def main():
    input_path=r"Z:\FacialDeformation_MPhys\paired_data\nina_abby\UIDQQ0Q00Q910\CT_WHOLE_CNS_cropped.nii"
    output_path= r"Z:\FacialDeformation_MPhys\paired_data\nina_abby"

    totalsegmentator(input_path,output_path,task = 'craniofacial_structures')
    print('\nDone!')

if __name__ == "__main__":
 main()

In [ ]:
# === Cell 3: Ensemble fusion and evaluation ===

from medpy.metric import binary

def read_mask(path):
    return sitk.GetArrayFromImage(sitk.ReadImage(str(path)))

def dice(a, b):
    inter = np.logical_and(a, b).sum()
    return 2.0 * inter / (a.sum() + b.sum() + 1e-8)

def volume_mm3(mask, spacing):
    return mask.sum() * np.prod(spacing)

# Define where the completed TotalSegmentator runs live
RUNS = [
    Path(BASE_PATH) / "TS_run1",
    Path(BASE_PATH) / "TS_run2",
    Path(BASE_PATH) / "TS_run3",
]

records = []

# Iterate over cases
case_ids = [d.name for d in RUNS[0].iterdir() if d.is_dir()]

for case_id in tqdm(case_ids):
    all_struct_masks = {s: [] for s in STRUCTURES}
    spacing = None

    # load all masks
    for run in RUNS:
        case_folder = run / case_id
        for s in STRUCTURES:
            fpath = case_folder / f"{s}.nii.gz"
            if fpath.exists():
                img = sitk.ReadImage(str(fpath))
                spacing = img.GetSpacing()
                all_struct_masks[s].append(read_mask(fpath))

    for s in STRUCTURES:
        if len(all_struct_masks[s]) < 2:
            continue
        arrs = np.stack(all_struct_masks[s])

        # Majority vote -> pseudo ground truth
        pseudo_gt = (arrs.sum(axis=0) >= (len(arrs) / 2)).astype(np.uint8)

        # Compute per-run Dice to pseudo-GT + volume
        dscs = [dice(a, pseudo_gt) for a in arrs]
        vols = [volume_mm3(a, spacing) for a in arrs]

        record = {
            "case_id": case_id,
            "structure": s,
            "mean_dice_vs_pseudo": np.mean(dscs),
            "std_dice_vs_pseudo": np.std(dscs),
            "mean_volume_mm3": np.mean(vols),
            "std_volume_mm3": np.std(vols),
            "num_runs": len(arrs)
        }
        records.append(record)

df = pd.DataFrame(records)
df.to_csv(OUTDIR / "ensemble_evaluation_summary.csv", index=False)
print("✅ Saved:", OUTDIR / "ensemble_evaluation_summary.csv")


In [ ]:
# === Cell 4: Quick visualization ===
import matplotlib.pyplot as plt

for s in STRUCTURES:
    subset = df[df["structure"] == s]
    plt.figure()
    plt.title(f"{s} — Dice stability across runs")
    plt.boxplot(subset["mean_dice_vs_pseudo"], vert=False)
    plt.xlabel("Dice vs pseudo-GT")
    plt.grid(True)
    plt.show()

df.groupby("structure")[["mean_dice_vs_pseudo", "std_dice_vs_pseudo", "mean_volume_mm3"]].mean()
